# Real-World Use Case: Financial Portfolio Analysis

This notebook demonstrates a **complete real-world workflow** combining all Trelliscope features.

## Scenario: Portfolio Manager Dashboard

You manage a diversified portfolio of 60 assets across:
- **Asset Classes**: Stocks, Bonds, Commodities, Real Estate, Crypto
- **Regions**: North America, Europe, Asia, Emerging Markets
- **Risk Profiles**: Conservative, Moderate, Aggressive

**Goals**:
1. Monitor performance across all assets
2. Identify underperformers and star performers
3. Analyze risk-return profiles
4. Track sector correlations
5. Generate client reports

## Features Demonstrated (Complete Integration)

- ✅ **Plotly Interactive Charts** - Hover for detailed metrics
- ✅ **Mixed Visualizations** - Plotly candlesticks + matplotlib heatmaps
- ✅ **Advanced Filtering** - Multi-dimensional asset screening
- ✅ **Keyboard Navigation** - Rapid portfolio scanning
- ✅ **Saved Views** - "Top Performers", "High Risk", "Sector Analysis"
- ✅ **CSV Export** - Client reports and detailed analysis
- ✅ **Panel Details Modal** - Full-size chart analysis

## Generated Visualizations

Each asset gets:
1. **Plotly Candlestick**: Interactive price history with volume
2. **Matplotlib Returns Dist**: Statistical analysis with Q-Q plot
3. Comprehensive metrics: Sharpe ratio, max drawdown, volatility, etc.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pathlib import Path
import sys
from datetime import datetime, timedelta
from scipy import stats

sys.path.insert(0, str(Path.cwd().parent.parent))

from trelliscope import Display
from trelliscope.dash_viewer import create_dash_app

plt.style.use('seaborn-v0_8-darkgrid')
np.random.seed(42)

## Generate Realistic Financial Data

In [ ]:
# Asset universe
asset_classes = ['Stocks', 'Bonds', 'Commodities', 'Real Estate', 'Crypto']
regions = ['North America', 'Europe', 'Asia', 'Emerging Markets']
risk_profiles = ['Conservative', 'Moderate', 'Aggressive']

n_assets = 60
n_days = 252  # Trading year

print(f"Generating {n_assets} assets with {n_days} days of price history...\n")

start_date = datetime(2024, 1, 1)
dates = [start_date + timedelta(days=i) for i in range(n_days)]

portfolio_data = []

for i in range(n_assets):
    ticker = f'ASSET{i+1:02d}'
    asset_class = asset_classes[i % len(asset_classes)]
    region = regions[i % len(regions)]
    
    # Risk characteristics vary by asset class
    if asset_class == 'Stocks':
        volatility = np.random.uniform(0.15, 0.35)
        expected_return = np.random.uniform(0.08, 0.20)
    elif asset_class == 'Bonds':
        volatility = np.random.uniform(0.03, 0.10)
        expected_return = np.random.uniform(0.02, 0.06)
    elif asset_class == 'Commodities':
        volatility = np.random.uniform(0.20, 0.40)
        expected_return = np.random.uniform(0.00, 0.15)
    elif asset_class == 'Real Estate':
        volatility = np.random.uniform(0.10, 0.20)
        expected_return = np.random.uniform(0.05, 0.12)
    else:  # Crypto
        volatility = np.random.uniform(0.40, 0.80)
        expected_return = np.random.uniform(-0.10, 0.50)
    
    # Generate price series using geometric Brownian motion
    dt = 1/252  # Daily
    drift = (expected_return - 0.5 * volatility**2) * dt
    shock = volatility * np.sqrt(dt) * np.random.randn(n_days)
    
    log_returns = drift + shock
    returns = np.exp(log_returns)
    
    initial_price = np.random.uniform(50, 500)
    prices = initial_price * np.cumprod(returns)
    
    # Generate OHLCV data
    highs = prices * (1 + np.abs(np.random.randn(n_days) * 0.02))
    lows = prices * (1 - np.abs(np.random.randn(n_days) * 0.02))
    opens = np.roll(prices, 1)
    opens[0] = prices[0]
    volumes = np.random.lognormal(15, 1, n_days)
    
    # Calculate metrics
    daily_returns = np.diff(prices) / prices[:-1]
    
    total_return = (prices[-1] - prices[0]) / prices[0] * 100
    ann_return = ((prices[-1] / prices[0]) ** (252 / n_days) - 1) * 100
    ann_volatility = np.std(daily_returns) * np.sqrt(252) * 100
    sharpe_ratio = (ann_return - 2.0) / ann_volatility if ann_volatility > 0 else 0  # Risk-free = 2%
    
    # Max drawdown
    cummax = np.maximum.accumulate(prices)
    drawdown = (prices - cummax) / cummax * 100
    max_drawdown = np.min(drawdown)
    
    # Risk profile
    if sharpe_ratio > 1.0 and ann_volatility < 15:
        risk_profile = 'Conservative'
    elif sharpe_ratio > 0.5 and ann_volatility < 25:
        risk_profile = 'Moderate'
    else:
        risk_profile = 'Aggressive'
    
    # Performance tier
    if total_return > 20:
        tier = 'Star'
    elif total_return > 5:
        tier = 'Good'
    elif total_return > -5:
        tier = 'Neutral'
    else:
        tier = 'Underperformer'
    
    portfolio_data.append({
        'ticker': ticker,
        'asset_class': asset_class,
        'region': region,
        'risk_profile': risk_profile,
        'performance_tier': tier,
        'total_return': total_return,
        'ann_return': ann_return,
        'ann_volatility': ann_volatility,
        'sharpe_ratio': sharpe_ratio,
        'max_drawdown': max_drawdown,
        'current_price': prices[-1],
        # Store data for charts
        'dates': dates,
        'prices': prices,
        'opens': opens,
        'highs': highs,
        'lows': lows,
        'volumes': volumes,
        'returns': daily_returns
    })
    
    if (i + 1) % 15 == 0:
        print(f"  Generated {i+1}/{n_assets} assets...")

print(f"\n✓ Generated {len(portfolio_data)} assets with complete price history")

In [ ]:
# Quick stats
df_temp = pd.DataFrame([{k: v for k, v in asset.items() if k not in ['dates', 'prices', 'opens', 'highs', 'lows', 'volumes', 'returns']} 
                        for asset in portfolio_data])

print("Portfolio Summary:\n")
print(f"By Asset Class:")
print(df_temp.groupby('asset_class')['total_return'].agg(['count', 'mean', 'std']).round(2))

print(f"\nBy Performance Tier:")
print(df_temp['performance_tier'].value_counts())

## Create Visualization Functions

In [ ]:
def create_plotly_candlestick(asset_data):
    """
    Interactive Plotly candlestick chart with volume.
    
    Features: Hover details, zoom, pan, range selector
    """
    ticker = asset_data['ticker']
    dates = asset_data['dates']
    
    fig = make_subplots(
        rows=2, cols=1,
        row_heights=[0.7, 0.3],
        vertical_spacing=0.05,
        subplot_titles=('Price', 'Volume'),
        shared_xaxes=True
    )
    
    # Candlestick
    fig.add_trace(
        go.Candlestick(
            x=dates,
            open=asset_data['opens'],
            high=asset_data['highs'],
            low=asset_data['lows'],
            close=asset_data['prices'],
            name='Price',
            increasing_line_color='#26a69a',
            decreasing_line_color='#ef5350'
        ),
        row=1, col=1
    )
    
    # Volume bars
    colors = ['#26a69a' if asset_data['prices'][i] >= asset_data['opens'][i] else '#ef5350' 
              for i in range(len(dates))]
    
    fig.add_trace(
        go.Bar(
            x=dates,
            y=asset_data['volumes'],
            name='Volume',
            marker_color=colors,
            showlegend=False
        ),
        row=2, col=1
    )
    
    # Add performance metrics as annotation
    metrics_text = (
        f"Return: {asset_data['total_return']:.1f}% | "
        f"Volatility: {asset_data['ann_volatility']:.1f}% | "
        f"Sharpe: {asset_data['sharpe_ratio']:.2f}"
    )
    
    fig.add_annotation(
        text=metrics_text,
        xref='paper', yref='paper',
        x=0.5, y=1.12,
        showarrow=False,
        font=dict(size=11, color='#666'),
        align='center'
    )
    
    fig.update_layout(
        title=f"{ticker} - {asset_data['asset_class']} ({asset_data['region']})",
        xaxis_rangeslider_visible=False,
        height=550,
        hovermode='x unified',
        plot_bgcolor='white',
        paper_bgcolor='white'
    )
    
    fig.update_xaxes(showgrid=True, gridcolor='#e0e0e0')
    fig.update_yaxes(showgrid=True, gridcolor='#e0e0e0')
    
    return fig


def create_matplotlib_returns_analysis(asset_data):
    """
    Statistical returns analysis with matplotlib.
    
    Panels: Returns distribution, Q-Q plot, rolling volatility, drawdown
    """
    ticker = asset_data['ticker']
    returns = asset_data['returns'] * 100  # Convert to %
    dates = asset_data['dates'][1:]  # Returns are 1 day shorter
    prices = asset_data['prices']
    
    fig = plt.figure(figsize=(12, 9))
    gs = gridspec.GridSpec(2, 2, hspace=0.3, wspace=0.3)
    
    # 1. Returns histogram with normal fit
    ax1 = fig.add_subplot(gs[0, 0])
    ax1.hist(returns, bins=40, alpha=0.7, color='steelblue', edgecolor='black', density=True)
    
    # Fit normal distribution
    mu, sigma = returns.mean(), returns.std()
    x = np.linspace(returns.min(), returns.max(), 100)
    ax1.plot(x, stats.norm.pdf(x, mu, sigma), 'r-', linewidth=2, label=f'Normal(μ={mu:.2f}, σ={sigma:.2f})')
    
    ax1.set_xlabel('Daily Return (%)')
    ax1.set_ylabel('Density')
    ax1.set_title('Returns Distribution')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # 2. Q-Q plot
    ax2 = fig.add_subplot(gs[0, 1])
    stats.probplot(returns, dist="norm", plot=ax2)
    ax2.set_title('Q-Q Plot (Normality Test)')
    ax2.grid(True, alpha=0.3)
    
    # 3. Rolling volatility
    ax3 = fig.add_subplot(gs[1, 0])
    rolling_vol = pd.Series(returns).rolling(window=20).std() * np.sqrt(252)
    ax3.plot(dates, rolling_vol, linewidth=1.5, color='darkorange')
    ax3.axhline(asset_data['ann_volatility'], color='red', linestyle='--', linewidth=1, label=f'Annual: {asset_data["ann_volatility"]:.1f}%')
    ax3.set_xlabel('Date')
    ax3.set_ylabel('Annualized Volatility (%)')
    ax3.set_title('Rolling 20-Day Volatility')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    ax3.tick_params(axis='x', rotation=45)
    
    # 4. Drawdown chart
    ax4 = fig.add_subplot(gs[1, 1])
    cummax = np.maximum.accumulate(prices)
    drawdown = (prices - cummax) / cummax * 100
    
    ax4.fill_between(asset_data['dates'], drawdown, 0, color='red', alpha=0.3)
    ax4.plot(asset_data['dates'], drawdown, color='darkred', linewidth=1.5)
    ax4.axhline(asset_data['max_drawdown'], color='red', linestyle='--', linewidth=1, 
                label=f'Max DD: {asset_data["max_drawdown"]:.1f}%')
    ax4.set_xlabel('Date')
    ax4.set_ylabel('Drawdown (%)')
    ax4.set_title('Underwater Chart')
    ax4.legend()
    ax4.grid(True, alpha=0.3)
    ax4.tick_params(axis='x', rotation=45)
    
    fig.suptitle(f'{ticker} - Statistical Analysis', fontsize=14, fontweight='bold', y=0.995)
    
    return fig


print("✓ Created visualization functions (Plotly + Matplotlib)")

## Generate All Panels

Each asset gets 2 panels:
1. Plotly candlestick (interactive)
2. Matplotlib statistical analysis (static)

In [ ]:
print(f"Generating {n_assets * 2} panels (candlestick + returns analysis)...\n")

final_data = []

for i, asset in enumerate(portfolio_data):
    # Common metadata
    base_data = {
        'ticker': asset['ticker'],
        'asset_class': asset['asset_class'],
        'region': asset['region'],
        'risk_profile': asset['risk_profile'],
        'performance_tier': asset['performance_tier'],
        'total_return': asset['total_return'],
        'ann_return': asset['ann_return'],
        'ann_volatility': asset['ann_volatility'],
        'sharpe_ratio': asset['sharpe_ratio'],
        'max_drawdown': asset['max_drawdown'],
        'current_price': asset['current_price']
    }
    
    # Plotly candlestick
    plotly_fig = create_plotly_candlestick(asset)
    final_data.append({
        **base_data,
        'chart_type': 'Candlestick',
        'format': 'Interactive (Plotly)',
        'panel': plotly_fig
    })
    
    # Matplotlib statistical
    mpl_fig = create_matplotlib_returns_analysis(asset)
    final_data.append({
        **base_data,
        'chart_type': 'Statistical Analysis',
        'format': 'Static (Matplotlib)',
        'panel': mpl_fig
    })
    plt.close(mpl_fig)
    
    if (i + 1) % 15 == 0:
        print(f"  Generated {(i+1)*2}/{n_assets*2} panels...")

final_df = pd.DataFrame(final_data)
print(f"\n✓ Generated {len(final_df)} panels")
print(f"  - Plotly (interactive): {len(final_df[final_df['format'] == 'Interactive (Plotly)'])}")
print(f"  - Matplotlib (static): {len(final_df[final_df['format'] == 'Static (Matplotlib)'])}")

## Create Portfolio Display

In [ ]:
display = (
    Display(
        final_df,
        name="financial_portfolio_dashboard",
        description="Complete Portfolio Analysis - 60 Assets with Interactive Candlesticks & Statistical Analysis"
    )
    .set_panel_column("panel")
    .infer_metas()
    .set_default_layout(ncol=2, nrow=2)  # 4 panels per page
    .set_default_labels(["ticker", "asset_class", "performance_tier", "sharpe_ratio"])
)

print("✓ Portfolio display configured")
print(f"  Assets: {n_assets}")
print(f"  Total panels: {len(final_df)}")
print(f"  Asset classes: {final_df['asset_class'].nunique()}")
print(f"  Performance tiers: {final_df['performance_tier'].nunique()}")

## Launch Portfolio Dashboard

### 🎯 Complete Workflow: Portfolio Review

#### **Morning Portfolio Check** (5 minutes)

**1. Star Performers View**
```
Filter: performance_tier = "Star"
Filter: chart_type = "Candlestick"
Sort: total_return (descending)
Save View: "Morning - Top Performers"
```
→ Quick scan of best assets with interactive charts

**2. Risk Assessment**
```
Filter: performance_tier = "Underperformer"
Filter: chart_type = "Statistical Analysis"
Sort: max_drawdown (ascending)
Save View: "Morning - Risk Check"
```
→ Review statistical diagnostics for underperformers

**3. High Volatility Assets**
```
Filter: ann_volatility > 30
Filter: chart_type = "Candlestick"
Save View: "High Vol Monitoring"
```

---

#### **Client Reporting** (10 minutes)

**Conservative Portfolio Report**
```
Filter: risk_profile = "Conservative"
Filter: chart_type = "Candlestick"
Export CSV → conservative_holdings.csv
Open in Excel → Client presentation
```

**Sector Analysis - Technology**
```
Search: "Stocks" (global search)
Filter: region = "North America"
Sort: sharpe_ratio (descending)
Export View JSON → tech_sector_config.json
Share config with team
```

---

#### **Deep Dive Analysis** (15 minutes)

**1. Interactive Exploration**
```
- Load "Star Performers" view
- Click any candlestick panel → Full-size modal
- Hover over candles → Detailed OHLC data
- Zoom into recent volatility spike
- Press → to see next panel in modal
```

**2. Statistical Verification**
```
- Filter: Same ticker
- Filter: chart_type = "Statistical Analysis"
- Examine Q-Q plot for normality
- Check drawdown chart
- Review rolling volatility trends
```

**3. Comparative Analysis**
```
Filter: asset_class = "Crypto"
Sort: sharpe_ratio (descending)
→ → → Navigate pages to see all crypto assets
+ + + Increase grid to see more at once
```

---

### 💼 Pre-Configured Views (Create These)

1. **"Daily Dashboard"**
   - All star + good performers
   - Candlestick charts only
   - Sorted by total return

2. **"Risk Review"**
   - All aggressive + underperformers
   - Statistical analysis charts
   - Sorted by max drawdown

3. **"Emerging Markets"**
   - Region = "Emerging Markets"
   - All chart types
   - Sorted by sharpe ratio

4. **"Crypto Watch"**
   - Asset class = "Crypto"
   - Candlestick only
   - Sorted by ann_volatility

5. **"Conservative Holdings"**
   - Risk profile = "Conservative"
   - Statistical analysis
   - Export ready for client reports

---

### 🚀 Power User Shortcuts

```
/ → Type "Star"           Quick filter to star performers
→ → → → →                 Rapid page scanning
End                       Jump to end of results
Ctrl+S                    Save current view
+ + +                     Increase grid to 3×3
Click panel               Full-size interactive analysis
Ctrl+R                    Reset to start fresh
```

---

### 📊 Expected Insights

From this portfolio, you should discover:
- **Star performers**: High Sharpe ratio assets worth increasing allocation
- **Underperformers**: Assets with negative returns or poor risk-adjusted returns
- **Volatility clusters**: Crypto and commodities will show higher ann_volatility
- **Drawdown patterns**: Some assets recovered quickly, others still underwater
- **Regional differences**: Emerging markets may show higher risk but mixed returns

---

The viewer opens at http://localhost:8058

**This is your complete portfolio management command center!**

In [ ]:
app = create_dash_app(display)
app.run(debug=False, host='127.0.0.1', port=8058)